# Tuning the Top-k Query parameter

First we set up the environment

In [2]:
from dotenv import load_dotenv
import os
import psycopg
import pandas as pd
import numpy as np
from time import time
import torch
from TextEnrichers import get_enricher, TextEnricher
from database.database import Database
from Embedders import Embedder, get_embedder
from tqdm import tqdm

load_dotenv('.env', override=True)
print(os.getenv('DB_PORT'))


5432


In [3]:
# Database setup
db = Database()
db.test_connection()

device = 'cuda' if torch.cuda.is_available(
) else 'mps' if torch.mps.is_available() else 'cpu'
print(f"Using device: {device}")

=================================CONFIG=================================
Database         User             Host                             Port            
citeline_db      bbasseri         localhost                        5432            
Database version: ('PostgreSQL 17.3 (Homebrew) on x86_64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit',)
Using device: mps


## Investigating precision over `k`

For our various embedding models and enrichment strategies, we want to know the smallest `top_k` value that will still retrieve the target reference for a given sentence. 

To investigate this, we'll sample 100 examples from the non-trivial training data. Each example typically has 1-2 target DOI's. For each example, we'll query the database with a large `top_k` parameter to start, so we can be sure the database returns the target references. Then we can ask at what index in the query results does a target DOI first appear. Ideally, the ranks will all be very high, indicated by having *low* indices in the query results. We also expect enriched examples to have their target doi's higher ranked (lower indices).

In [ ]:
data = pd.read_json('data/dataset/split/train.jsonl', lines=True)
examples = data.sample(100, random_state=42)
examples.head()


def lowest_index_matching_doi(target_doi: str, query_results: list) -> int:
    """
    Returns the first index of the query results where the chunk doi matches the target doi.
    If no match is found, returns -1.
    """
    for i, result in enumerate(query_results):
        if target_doi == result.doi:
            return i
    return -1


def get_ranks(
    example: pd.Series,
    embedding,
    # embedder: Embedder,
    # enricher: TextEnricher,
    table_name: str,
    top_k: int,
    ef_search: int,
    metric: str = 'vector_cosine_ops'
) -> list[int]:
    target_dois = example['citation_dois']

    # Query
    start = time()
    query_results = db.query_vector_table(
        query_vector=embedding,
        target_column='bge_norm',
        table_name='library',
        top_k=top_k,
        use_index=True,
        ef_search=ef_search
    )
    
    ranks = [lowest_index_matching_doi(
        target_doi=doi, query_results=query_results) for doi in target_dois]
    return ranks

In [9]:
from itertools import product


embedding_models = ['BAAI/bge-small-en']
if device == 'cuda':
    embedding_models += ['bert-base-uncased', 'adsabs/astroBERT']
combos = list(product(embedding_models, TextEnricher.ENRICHMENT_FN.keys()))
print(f"Combos: {combos}")

Combos: [('BAAI/bge-small-en', 'identity'), ('BAAI/bge-small-en', 'add_abstract'), ('BAAI/bge-small-en', 'add_title'), ('BAAI/bge-small-en', 'add_title_and_abstract')]


In [13]:
# Dict mapping "model name": pd.DataFrame() of results (enrichment functions used -> columns)
results = {model: pd.DataFrame() for model in embedding_models}
top_k = 5000

for embedding_model, enricher_name in combos:
    print(f"Embedding model: {embedding_model}, Enricher: {enricher_name}")
    embedder = get_embedder(embedding_model, device=device)
    
    # Enrich
    enricher = get_enricher(enricher_name)
    texts_with_dois = list(
        examples[['sent_no_cit', 'source_doi']].itertuples(index=False, name=None))
    enriched_texts = enricher.enrich_batch(texts_with_dois)


    embeddings = embedder(enriched_texts)
    ranks = []
    for i in tqdm(range(len(examples))):
        embedding = embeddings[i]
        example = examples.iloc[i]
        ranks += get_ranks(
            example=example,
            embedding=embedding,
            # embedder=embedder,
            # enricher=enricher,
            table_name='library',
            top_k=top_k,
            ef_search=top_k)
    series = pd.Series(ranks)
    results[embedding_model][enricher_name] = series

model_names_to_filenames = {
    'BAAI/bge-small-en': 'bge',
    'bert-base-uncased': 'bert',
    'adsabs/astroBERT': 'astrobert'
}

# Save the results to CSV files
for model_name, df in results.items():
    filename = f"tests/{model_names_to_filenames[model_name]}_ranks_topk{top_k}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved results for {model_name} to {filename}")

Embedding model: BAAI/bge-small-en, Enricher: identity


  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:00<01:35,  1.04it/s]

  Query execution time: 0.78 seconds


  2%|▏         | 2/100 [00:01<01:06,  1.47it/s]

...

  Query execution time: 0.11 seconds
  Query execution time: 0.09 seconds


 98%|█████████▊| 98/100 [00:15<00:00,  8.79it/s]

  Query execution time: 0.09 seconds
  Query execution time: 0.10 seconds


100%|██████████| 100/100 [00:15<00:00,  6.35it/s]

  Query execution time: 0.10 seconds
  Query execution time: 0.11 seconds
Saved results for BAAI/bge-small-en to tests/bge_ranks_topk5000.csv


Saved results for BAAI/bge-small-en to tests/bge_ranks_topk500.csv
